In [1]:
from primaite.session.session import PrimaiteSession
from primaite.game.game import PrimaiteGame
from primaite.config.load import example_config_path

from primaite.simulator.system.services.database.database_service import DatabaseService

import yaml

/home/cade/repos/PrimAITE/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-26 23:25:47,985	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-26 23:25:51,213	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-26 23:25:51,491	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] 

In [2]:

with open(example_config_path(),'r') as cfgfile:
    cfg = yaml.safe_load(cfgfile)
game = PrimaiteGame.from_config(cfg)
net = game.simulation.network
database_server = net.get_node_by_hostname('database_server')
web_server = net.get_node_by_hostname('web_server')
client_1 = net.get_node_by_hostname('client_1')

db_service = database_server.software_manager.software["DatabaseService"]
db_client = web_server.software_manager.software["DatabaseClient"]
# db_client.run()
db_manipulation_bot = client_1.software_manager.software["DataManipulationBot"]
db_manipulation_bot.port_scan_p_of_success=1.0
db_manipulation_bot.data_manipulation_p_of_success=1.0


2023-11-26 23:25:51,579::ERROR::primaite.simulator.network.hardware.base::175::NIC a9:92:0a:5e:1b:e4/127.0.0.1 cannot be enabled as it is not connected to a Link
2023-11-26 23:25:51,580::ERROR::primaite.simulator.network.hardware.base::175::NIC ef:03:23:af:3c:19/127.0.0.1 cannot be enabled as it is not connected to a Link
2023-11-26 23:25:51,581::ERROR::primaite.simulator.network.hardware.base::175::NIC ae:cf:83:2f:94:17/127.0.0.1 cannot be enabled as it is not connected to a Link
2023-11-26 23:25:51,582::ERROR::primaite.simulator.network.hardware.base::175::NIC 4c:b2:99:e2:4a:5d/127.0.0.1 cannot be enabled as it is not connected to a Link
2023-11-26 23:25:51,583::ERROR::primaite.simulator.network.hardware.base::175::NIC b9:eb:f9:c2:17:2f/127.0.0.1 cannot be enabled as it is not connected to a Link
2023-11-26 23:25:51,590::ERROR::primaite.simulator.network.hardware.base::175::NIC cb:df:ca:54:be:01/192.168.1.10 cannot be enabled as it is not connected to a Link
2023-11-26 23:25:51,595::

installing DNSServer on node domain_controller
installing DatabaseClient on node web_server
installing WebServer on node web_server
installing DatabaseService on node database_server
installing FTPClient on node database_server
installing FTPServer on node backup_server
installing DNSClient on node client_1
installing DNSClient on node client_2


In [3]:
db_client.run()

In [4]:
db_service.backup_database()

True

In [5]:
db_client.query("SELECT")

True

In [6]:
db_manipulation_bot.run()

In [7]:
db_client.query("SELECT")

False

In [8]:
db_service.restore_backup()

True

In [9]:
db_client.query("SELECT")

True

In [ ]:
db_manipulation_bot.run()

In [ ]:
client_1.ping(database_server.ethernet_port[1].ip_address)

In [10]:
from pydantic import validate_call, BaseModel

In [11]:
class A(BaseModel):
    x:int

    @validate_call
    def increase_x(self, by:int) -> None:
        self.x += 1

In [12]:
my_a = A(x=3)

In [15]:
my_a.increase_x(3.2)

ValidationError: 1 validation error for increase_x
0
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=3.2, input_type=float]
    For further information visit https://errors.pydantic.dev/2.1/v/int_from_float